# 발표 소단원 리스트

(4, 13)
(5, 1)
(5, 5)
(5, 9)
(5, 13)
(5, 17)
(5, 21)
(6, 4)

# 전체 요약자료

- 4.13 : 

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 4.13 데이터 처리 파이프라인 생성
- 데이터 처리를 데이터 처리 파이프라인과 같은 방식으로 순차적으로 처리하고 싶을 때
 
 ex) 처리해야 할 방대한 데이터가 있지만 메모리에 한꺼번에 들어가지 않는 경우
 
 -> 제너레이터 함수 사용

In [ ]:
import os
import fnmatch
import gzip
import bz2
import re

def get_find(filepat, top):
    '''
    디렉터리 트리에서 와일드카드 패턴에 매칭하는 모든 파일 이름을 찾음
    '''
    # os.walk는 시작 디렉터리부터 시작하여 그 하위의 모든 디렉터리를 차례대로 방문하게 해주는 함수
    for path, dirlist, filelist in os.walk(top): 
        
        # fnmatch.filter(names, pattern): pattern에 일치하는 names 리스트의 부분 집합을 반환
        for name in fnmatch.filter(filelist, filepat):
            
            # os.path.join('C:\Tmp', 'a', 'b') : "C:\Tmp\a\b" 경로를 합침
            yield os.path.join(path, name)
            
def gen_opener(filenames):
    '''
    파일 이름 시퀀스를 하나씩 열어 파일 객체를 생성한다.
    다음 순환으로 넘어가는 순간 파일을 닫음
    '''
    for filename in filenames :
        if filename.endswitch('.gz'):
            f = gzip.open(filename, 'rt')
        elif filename.endswitch('.bz2'):
            f = bz2.open(filename, 'rt')
        else :
            f = open(filename, 'rt')
        yield f
        f.close()
        
#목적 : 입력받은 시퀀스를 하나로 합치는 것, 
#itemtools.chain() 함수가 비슷한 기능. 
# yield from 문은 gen_concatenat()가 제너레이터 it이 생성한 모든 값을 분출하도록 만듦.
def get_concatenate(iterators):
    '''
    이터레이터 시퀀스를 합쳐 하나의 시퀀스로 만든다
    '''
    for it in iterators:
        yield from it
        
def gen_grep(pattern, lines):
    '''
    라인 시퀀스에서 정규식 패턴을 살펴봄
    '''
    pat = re.compile(pattern)
    for line in lines:
        if pat.search(line):
            yield line

python 이란 단어를 포함하고 있는 모든 로그 라인을 찾으려 할 때 :

In [ ]:
lognames = gen_find('access-log*', 'www')
files = gen_opener(lognames)
lines = gen_concatenate(files)
pylines = gen_grep('(?i)python', lines)
for line in pylines:
    print(line)

yield 문 : 데이터 생성자

for 문 : 데이터 소비자

제너레이터가 쌓이면, 각 yield가 순환을 하며 데이터의 아이템 하나를 파이프라인의 다음 단계로 넘김.

-> 각 제너레이터 함수를 작게 모듈화할 수 있음.

-> 메모리 효율성 높음(아주 방대한 디렉터리와 파일에도 잘 동작함)
    

#### 보충

In [ ]:
# chain(*iterable)은 iterable한 객체들을 인수로 받아 하나의 iterator로 반환 
# 간단히 말하면 리스트( lists/tuples/iterables ) 를 연결하는 것

from itertools import chain
country = ['대한민국','스웨덴', '미국']
capital = ['서울','스톡홀름','워싱턴']
c = chain(country, capital)
next(c)
next(c)
next(c)
next(c)
next(c)

In [ ]:
files = gen_opener(lognames)
lines = itertools.chain(*files)

itertools.chain() 함수는 묶어 줄 모든 순환 객체를 인자로 전달해야 함.
위와 같은 코드에 쓰면 이 제너레이터는 다음 번 순환 단계에서 곧바로 닫는 파일 시퀀스를 생성하기 때문에 chain()을 사용할 수 없다.

- 더 많은 예제 : David Beazley의 "Generator Tricks for Systems Programmers" 튜토리얼 참고

https://www.dabeaz.com/generators/

https://www.dabeaz.com/generators/Generators.pdf

### 5.1 텍스트 데이터 읽고 쓰기
- 텍스트 데이터를 읽거나 써야 하는데 ASCII, UTF-8, UTF-16과 같이 서로 다른 인코딩을 사용해야 함


In [ ]:
# 파일 전체를 하나의 문자열로 읽음
with open('somefile.txt', 'rt') as f:
    data=f.read()
    
# 파일의 줄 순환
with open('somefile.txt', 'rt') as f:
    for line in f:
        # 라인 처리

In [ ]:
# 텍스트 데이터 쓰기, 기존 내용을 모두 지우고 새로운 내용을 덮어씀 (파일의 끝에 내용을 추가하려면 at 모드로 사용
with open('somefile.txt', 'wt') as f:
    f.write(text1)
    f.write(text2)
    
with open('somefile.txt', 'wt') as f:
    print(line1, file=f)
    print(line2, file=f)

    
#위와 같이 with 문을 사용하지 않으려면 반드시 파일을 닫아야 함.
f = open('somefile.txt', 'rt')
data = f.read()
f.close()

#### 보충
대부분의 컴퓨터는 기본 인코딩으로 utf-8  사용

읽고 쓸 텍스트가 다른 인코딩을 사용한다면 open()에 추가적인 encoding 인자 전달

with open('somefile.txt', 'rt', encoding='latin-1') as f:
    
    
일반적으로 사용하는 인코딩 : ascii, latin-1, utf-8, utf-16
- ascii : U+0000에서 U+007F 범위의 7비트 문자에 일치
- latin-1 : 0-255바이트를 유니코 U+0000에서 U+00FF 로 매핑. (인코딩의 종류를 모르는 텍스트를 읽을 때 절대 에러가 발생하지 않음)

    파일을 latin-1로 읽으면 완벽하게 올바른 텍스트 디코딩을 만들어내지 못할 수도 있음.
    
    여전히 유용한 데이터 추출 + 데이터를 다시 돌려보낼 때 원본 입력을 보전함.

- 줄바꿈 문자 Unix : \n , Window : \r\n
- 기본적으로 파이썬은 'universal newline' 모드로 동작 = 일반적인 모든 줄바꿈을 알아보고 읽을 때 모든 줄바꿈 문자를 \n로 변환.
출력시에는 줄바꿈 문자 \n을 시스템 기본 문자로 변환.

-> 이러한 자동 변환을 원하지 않으면 newline='' 인자 사용

ex)
with open('somefile.txt', 'rt', newline='') as f:

In [ ]:
#Unix 컴퓨터에서 WIndows 형식으로 인코딩된 텍스트 파일을 읽어볼 때
# 줄바꿈 변환 사용(기본) :
f = open('hello.txt', 'rt')
f.read() # 'hello world!\n'

# 줄바꿈 변환 미사용
g = open('hello.txt', 'rt', newline='')
g.read() # 'hello world!\r\n'

In [ ]:
# 인코딩으 올바르게 사용했는지 확인 후에도 에러가 사라지지 않는다면 : errors 인자 전달
g = open('hello.txt', 'rt', encoding='ascii', errors='ignore') # 알 수 없는 문자를 무시
g = open('hello.txt', 'rt', encoding='ascii', errors='replace') #알 수 없는 문자를 유티코드 U+fffd로 치환

### 5.5 존재하지 않는 파일에 쓰기
- 파일이 시스템에 존재하지 않을 때, 데이터를 파일에 쓰기 : open()에 x모드 사용

 x 모드는 파이썬 3의 확장 기능(이전 파이썬은 지원하지 않음)

In [ ]:
with open('somefile', 'wt') as f:
    f.write('Hello\n') ##'somefile'이 없으면 오류
    
with open('somefile', 'xt') as f:
    f.write('Hello\n')

In [ ]:
import os
if not os.path.exists('somefile'):
    with open('somefile', 'wt') as f:
        f.write('Hello\n')
        
else:
    print('File already exists!')

### 5.9 바이너리 데이터를 수정 가능한 버퍼에 넣기
- 바이너리 데이터를 읽어 수정 가능 버퍼에 넣을 때 어떠한 복사 과정도 거치고 싶지 않을 때

- readinto(b: bytes) ->  return int

: Read up to len(b) bytes from the object and stores them in b, returning the number of bytes read. Like .read, fewer than len(b) bytes may be read, and 0 indicates end of file. None is returned if a non-blocking object has no bytes available. The length of b is never changed.

readinto() 메소드를 사용해서 미리 할당해 높은 배열에 데이터를 채워 넣을 수 있음 (array 모듈이나 numpy와 같은 라이브러리를 사용해서 생성한 배열을 사용할 수도 있음)



read(num) :  num bytes를 받을 때까지 read / 새로운 객체를 할당하고 반환

readinto(buf) : 최대 len(buf) bytes를 읽고 반환 / 기존의 버퍼에 내용을 채워 넣음 -> 불필요한 메모리 할당을 피할 수 있음.

reads UP TO len(buf) bytes and returns, while read(num) will block until num bytes are received. 


In [ ]:
import os.path

def read_into_buffer(filename):
    buf = bytearray(os.path.getsize(filename)) #입력받은 경로에 대한 바이트 단위의 파일크기를 반환
    with open(filename, 'rb') as f:
        f.readinto(buf) #반환코드는 실제로 읽은 바이트 수, 이 경우엔 11 
    return buf 

In [ ]:
with open('sample.bin', 'wb') as f:
    f.write(b'Hello world')
    
buf = read_into_buffer('sample.bin')
buf
# result : bytearray(b'Hello world')

buf[0:5] = b'Hallo'
buf
# result : bytearray(b'Hallo world')

with open('newsample.bin', 'wb') as f:
    f.write(buf)


In [ ]:
# 레코드 크기가 고정적인 바이너리 파일을 읽을 때
record_size = 32

buf = bytearray(record_size)
with open('somefile', 'rb') as f:
    while True:
        n = f.readinto(buf)
        if n < record_size : #바이트 수가 제시한 버퍼의 크기보다 작다면 데이터에 이상이 있거나 무언가 잘려 나갔음을 의미
            break

#### 보충
- into 형식의 다른 함수 ex)recv_into(), pack_into()

: 직접 입출력 혹은 배열, 버퍼를 채우거나 수정하는 데 사용할 수 있도록 데이터에 대한 접근을 지원하는 것이 많음

- memoryview() :

: 외부의 자료를 읽어들일 때, 메모리에 복사 저장하는 것이 아닌 중간 단계인 버퍼에 저장
  
  버퍼에 저장된 데이터를 수정 가능
  
  bytes, bytearray 형식만 지원

In [ ]:
a = 'A'.encode() * 10
b = bytearray(a)
print(b)
buff_b = memoryview(b)
buff_b[9] = 66
print(b)

In [14]:
import time
a = 'A'.encode() * 10000
b = a
big = {}

print('memoryview 사용 :')
mem_b = memoryview(b)
start = time.time()
for abc in range(500000):
    big['b' + str(abc)] = mem_b[1:8000]
end = time.time()
print(end-start)

memoryview 사용 :
0.9194967746734619


In [15]:
a = 'A'.encode() * 10000
b = a
big = {}

print('사용 안함 :')
start2 = time.time()
for abc in range(500000):
    big['b' + str(abc)] = b[1:8000]
end2 = time.time()
print(end2-start2)

사용 안함 :
0.5967166423797607


### 5.13 디렉터리 리스팅 구하기
- 디렉터리나 파일 시스템 내부의 파일 리스트를 구하고 싶을 때

In [16]:
import os
names = os.listdir('somedir')
#디렉터리, 파일, 서브디렉터리, 심볼릭 링크 등 모든 것을 구할 수 있음

FileNotFoundError: [Errno 2] No such file or directory: 'somedir'

In [17]:
import os.path
#일반 파일 모두 구하기
names = [name for name in os.listdir('somedir')
        if os.path.isfile(os.path.join('somedir', name))]

#디렉터리 모두 구하기
dirnames = [name for name in os.listdir('somedir')
           if os.path.isdir(os.path.join('somedir', name))]

In [ ]:
#문자열의 startswith()와 endswith() 메소드를 사용하면 내용 걸러내기 유용
pyfiles = [name for name in os.listdir('somedir')
          if name.endswith('.py')]

In [ ]:
import glob
pyfiles = glob.glob('somedir/*.py')

from fnmatch import fnmatch
pyfiles = [name for name in os.listdir('somedir')
          if fnmatch(name, '*.py')]

#### 보충
- 리스트 + 파일 크기나 수정 날짜 등 메타데이터가 필요할 때

In [ ]:
import os
import os.path
import glob

pyfiles = glob.glob('*.py')

#파일 크기와 수정 날짜 구하기
name_sz_data = [(name, os.path.getsize(name), os.path.getmtime(name))
               for name in pyfiles]

for name, size, mtime in name_sz_data:
    print(name, size, mtime)

file_metadata = [(name, os.stat(name)) for name in pyfiles]
for name, meta in file_metadata:
    print(name, meta.st_size, meta,st_mtime)

-os.stat()

st_mode 파일 모드: 파일 유형 및 파일 모드 비트 (사용 권한).

st_dev : 이 파일이 있는 장치의 식별자.

st_nlink : 하드 링크 수.

st_uid : 파일 소유자의 사용자 식별자.

st_gid : 파일 소유자의 그룹 식별자.

st_size : 일반 파일 또는 심볼릭 링크면, 바이트 단위의 파일의 크기. 심볼릭 링크의 크기는 포함하고 있는 경로명의 길이이며, 끝나는 널 바이트는 포함하지 않습니다.

st_atime :  초 단위의 가장 최근의 액세스 시간.

st_mtime : 초 단위의 가장 최근의 내용 수정 시간.

st_ctime :  유닉스에서 가장 최근의 메타 데이터 변경 시간, 윈도우에서 생성 시간, 단위는 초.



### 5.17 텍스트 파일에 바이트 쓰기
- 텍스트 모드로 연 파일에 로우 바이트를 쓰고 싶을 때

In [19]:
import sys
#sys.stdout은 언제나 텍스트 모드로 열려있음
sys.stdout.write("aaa")
sys.stdout.write("bbb")

aaabbb

In [18]:
import sys
sys.stdout.write(b'Hello\n')

sys.stdout.buffer.write(b'Hello\n')

Hello


AttributeError: 'OutStream' object has no attribute 'buffer'

In [ ]:
텍스트 파일은 버퍼 바이너리 모드 파일 상단에 Unicode 인코딩/디코딩 레이어를 추가해서 생성

buffer 속성은 이 파일 아래 부분을 가리킴 -> 여기에 접근하면 텍스트 인코딩/디코딩 레이어를 우회할 수 있음.

### 5.21 파이썬 객체를 직렬화하기
- 파이썬 객체를 바이트 스트림에 직렬화시켜 파일이나 데이터베이스에 저장하거나 네트워크를 통해 전송
- pickle.dump() : 문자열이나 값을 한꺼번에 전달하는 기능
- pickle.load() : 바이트 스트림으로부터 객체를 다시 만들기 위해

대부분 파이썬 데이터 타입, 인스턴스, 사용자 정의 클래스와 잘 작동

파이썬 객체를 데이터베이스에 저장하거나 불러오고, 네트워크를 통해 전송하는 라이브러리를 사용한다면 내부적으로 pickle을 사용하고 있을 확률이 큼.

pickle은 파이썬에 특화된 데이터 인코딩으로서 자기 스스로를 설명

: 직렬화된 데이터에 객체의 시작과 끝 그리고 타입에 관한 정보가 포함되어 있음
    -> 레코드를 정의하지 않아도 잘 작동

In [ ]:
import pickle #가장 일반적인 접근

#pickle로 데이터를 저장하거나 불러올때는 파일을 바이트형식으로 읽거나 써야함 (wb, rb)
f = open('somefile', 'wb')
numbers = [1,2,3,4,5]
strings = ['Hello,', 'world!']

pickle.dump(32, f)
pickle.dump(numbers, f)
pickle.dump(strings, f)

f.close()

In [ ]:
f = open('somefile', 'rb')
num = pickle.load(f) #한 줄 씩 읽어옴
numbers = pickle.load(f)
strings = pickle.load(f)

print(num)
print(numbers)
print(strings)

f.close()

'''
32
[1,2,3,4,5]
['Hello,','world!']
'''

#### 보충
- pickle.load()는 믿을 수 없는 데이터에 절대 사용하면 안 된다. 로딩의 부작용으로 pickle이 자동으로 모듈을 불러오고 인스턴스를 만든다. 하지만 악의를 품은 사람이 이 동작을 잘못 사용하면 일종의 바이러스 코드를 만들어 파이썬이 자동으로 실행하도록 할 수 있다. 따라서 서로 인증을 거친 믿을 수 있는 시스템끼리 내부적으로만 pickle을 사용하는 것이 좋다.

- 피클할 수 없는 객체도 있음 (파일, 네트워크 연결, 스레드, 프로세스, 스택 프레임 등 외부 시스템 상태와 관련 있는 것들)
- 사용자 정의 클래스에 '__ getstate__()'와 '__ setstate__()' 메소드를 제공하면 이런 제약을 피할 수 있음
-> 정의를 했으면 pickle.dump()는 __ getstate__()를 호출해 피클할 수 있는 객체를 얻음
-> __ setstate__()는 언피클을 유발

- __ getstate__(self) : 객체의 __ dict__ 속성이 저장되는 대신 객체가 피클될 때 저장할 수 있는 사용자 정의 상태를 반환할 수 있음
- __ setstate__(self, state) : 객체가 언피클 되었을 때 __ setstate__가 정의 되었다면, 객체의 상태는 객체의 __ dict__에 직접 적용되지 않고 전달됨(__ getstate__와 함께 사용됨) 

In [1]:
import time
import threading

class Countdown :
    def __init__(self, n):
        self.n = n
        self.thr = threading.Thread(target=self.run)
        self.thr.daemon = True
        self.thr.start()
        
    def run(self):
        while self.n > 0:
            print('T-minus', self.n)
            self.n -= 1
            time.sleep(5)
            
    def __getstate__(self):
        return self.n
    
    def __setstate__(self, n):
        self.__init__(n)

In [3]:
c = Countdown(30)

T-minus 30
T-minus 29
T-minus 28
T-minus 27
T-minus 26
T-minus 25
T-minus 24
T-minus 23
T-minus 22
T-minus 21
T-minus 20
T-minus 19
T-minus 18
T-minus 17
T-minus 16
T-minus 15
T-minus 14
T-minus 13
T-minus 12
T-minus 11
T-minus 10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


In [ ]:
# c 실행하고 잠시 후에
f = open('cstate.p', 'wb')
import pickle
pickle.dump(c, f)
f.close()

In [4]:
# 파이썬 종료하고 재시작 후
f = open('cstate.p', 'rb')
pickle.load(f)

# 스레드가 다시 살아나서 처음으로 피클했을 때 종료했던 곳부터 시작함.

FileNotFoundError: [Errno 2] No such file or directory: 'cstate.p'

#### 보충
- pickle은 array 모듈이나 numpy와 같은 라이브러리가 만들어 낸 거대한 자료 구조에 사용하기에 효율적인 인코딩 방식이 아님

방대한 자료를 이동해야 한다면 벌크 배열을 파일에 저장하거나 HDF6와 같은 좀 더 표준화된 인코딩을 사용하는 것이 좋음.

- 오랜 기간 저장을 위해 pickle 사용은 지양하는 것이 좋음. (소스 코드에 변화가 잇으면 저장되어 있는 모든 데이터를 읽지 못하게 될 수 있음.)

- XML, CSV, JSON과 같은 좀 더 표준적인 데이터 인코딩을 사용하는 것이 더 좋음 (pickle보다 더 표준화되어 있고 지원하는 언어도 더 많으며 소스 코드가 변한다 해도 적응하기 더 쉬움)

### 6.4 매우 큰 XML 파일 증분 파싱하기
- 매우 큰 XML 파일에서 최소의 메모리만 사용하여 데이터를 추출하고 싶을 때

많은 메모리 (2GB 만)없이 컴퓨터에서 매우 큰 파일 (> 2GB)을 처리 할 수있는 파서를 작성

- iterparse() : 메소드를 사용하기 위해서는 파일 이름과 start, end, start-ns, end-ns 중 하나 이상을 포함한 이벤트 리스트를 넘겨 주어야 함.

iterparse()가 생성한 이터레이터는 (event, elem)으로 구성된 튜플을 만듦.

start 이벤트는 요소가 처음 생성되었지만 다른 데이터를 만들지 않았을 때 생성됨.

end 이벤트는 요소를 마쳤을 때 생성.

In [ ]:
>>> from xml.etree.ElementTree import iterparse
>>> data = iterparse('potholes.xml', ('start', 'end'))
>>> next(data)
('start', <Element 'data' at 0x7fba6aa9b638>)
>>> next(data)
('start', <Element 'country' at 0x7fba6a2474f8>)
>>> next(data)
('start', <Element 'rank' at 0x7fba6a251a48>)
>>> next(data)
('end', <Element 'rank' at 0x7fba6a251a48>)
>>> next(data)
('start', <Element 'year' at 0x7fba6a251a98>)
>>> next(data)
('end', <Element 'year' at 0x7fba6a251a98>)

In [ ]:
from xml.etree.ElementTree import iterparse

def parse_and_remove(filename, path):
    path_parts = path.split('/')
    
    doc = iterparse(filename, ('start', 'end'))
    #뿌리 요소 건너뛰기
    next(doc)
    
    tag_stack = []
    elem_stack = []
    for event, elem in doc:
        if event == 'start' :
            tag_stack.append(elem.tag)
            elem_stack.append(elem)
        elif event == 'end' :
            if tag_stack == path_parts:
                yield elem
                elem_stack[-2].remove(elem) ##메모리 절약 부분 : 앞에서 나온 요소를 부모로부터 제거하는 역할
            
            try:
                tag_stack.pop()
                elem_stack.pop()
                
            except IndexError:
                pass

#### 보충
- 문제점 : 실행 속도가 느림
- 문서 전체를 메모리에 읽어 놓고 실행하는 방식이 두 배 빠르지만 메모리 소비는 60배 많음

-> 메모리 / 실행 속도 중 우선시하는 것에 따라 다르게